# Care patient management system

In [2]:
import sqlite3 as sql
import pymongo
from pymongo import MongoClient
import ipywidgets as widgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox
from IPython.display import display, clear_output
import uuid
import pandas as pd
import numpy as np

# SQL Create - SQLite 3 

In [6]:
# Establish connection and configuration 
db = "care_db.db"
connection = sql.connect(db)
cur = connection.cursor()

In [130]:
# CREATE patient table with basic patient details
cur.execute('''CREATE TABLE patient (patient_id INTEGER PRIMARY KEY AUTOINCREMENT, first_name TEXT (20) NOT NULL, last_name TEXT (20) NOT NULL, dob DATE NOT NULL, room_no INTEGER(3) NOT NULL,emergency_contact BIGINT(14) NOT NULL, one_one_status BOOLEAN, enrolement_status BOOLEAN,details_id VARCHAR(40));''')

In [4]:
#CREATE carer table with care giver staffs details
cur.execute('''CREATE TABLE carer (carer_id INTEGER PRIMARY KEY AUTOINCREMENT, first_name TEXT (20) NOT NULL, last_name TEXT (20) NOT NULL, org_name TEXT (20))''')

In [7]:
#CREATE activities register 
cur.execute('''CREATE TABLE activities_register (activity_code INTEGER PRIMARY KEY AUTOINCREMENT, activity_name TEXT(25), activity_description TEXT(45))''')

In [9]:
#CREATE activities table for recording daily care_activities 
cur.execute('''CREATE TABLE activities (activity_id INTEGER PRIMARY KEY AUTOINCREMENT, carer_id INTEGER, patient_id INTEGER, activity_code INTEGER, date_time TEXT (25),  FOREIGN KEY (carer_id) REFERENCES carer (carer_id),  FOREIGN KEY (patient_id) REFERENCES patient (patient_id), FOREIGN KEY (activity_code) REFERENCES activities_register (activity_code))''')

In [10]:
connection.commit()
connection.close()

# NO SQL Create - Mongodb

In [15]:
# Establish connection and configuration 
mg=MongoClient('localhost', 27017)
mongo_db = mg.care_db_mongo #create new db / connect to existing

In [20]:
#create new collections
patient_details = mongo_db.details 
activity_log = mongo_db.activity 

# Insert a new patient 

Insert new patient details

In [4]:
#Define all our widgets
button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='info',
    tooltip='Confirm information and submmit',
    icon='check' 
)
f_name = widgets.Text(
placeholder='Type first name',
description='First Name :'
)
l_name = widgets.Text(
placeholder='Type last name',
description='Last Name :'
)
dob = widgets.DatePicker(
    description='Date of birth :',
    disabled=False
)
room = widgets.Text(
placeholder='Enter room number',
description='Room no :'
)
ph_no = widgets.Text(
placeholder='Enter UK phone number',
description='Emergency contact:'
)
one_to_one = widgets.Dropdown(
    options=['Yes', 'No'],
    value='No',
    description='One-One:',
    disabled=False,
)


l = Layout(flex='0 1 auto', height='100px', min_height='40px', width='auto')
care_plan = Textarea(value='', layout=l)
schedule = Textarea(value='', layout=l)
medical_history = Textarea(value='', layout=l)
vb = VBox([schedule,care_plan,medical_history], layout=Layout(flex='1 1 auto', width='auto'))

#Display all our widgets
display( 
widgets.HTML(
    value="<h4><b>Enter patient details here</b></h4>",
),f_name,l_name,dob,room,ph_no,one_to_one,widgets.HTML(
    value="<h4><b>Enter patient Schedule here</b></h4>",
),schedule,widgets.HTML(
    value="<h4><b>Enter patient Care Plan here</b></h4>",
),care_plan, widgets.HTML(
    value="<h4><b>Enter patient Medical history here</b></h4>",
),medical_history,button)  

#validation functions that could be expanded in future
class vald:
  def __init__(self, f_name, l_name, dob, room, ph_no, one_to_one):
    self.f_name = f_name
    self.l_name = l_name
    self.dob = dob
    self.room = room
    self.ph_no = ph_no
    self.one_to_one = one_to_one
    
  def validate(self):
    if (self.f_name == '') or (self.l_name == '') or (self.dob == None) or (self.room == '') or (self.ph_no == ''):
        print('Please make sure no fields are empty')
        return 1
    elif (str.isalpha(self.f_name) == False) or (str.isalpha(self.l_name) == False):
        print('Please make sure name only contains alphabets')
        return 1
    elif (str.isdigit(self.room) == False) or (str.isdigit(self.ph_no) == False):
        print('Please make sure room number only contains digits')
        return 1
    elif int(self.room)>10000:
        print('Please ensure that the room number is under 10000')
        return 1
    elif int(self.ph_no)>9999999999999:
        print('Please ensure that the phone number is a valid UK number')
        return 1
    else:
        return 0


#function for button click
def button_click(click):
    #Validation
    p1 = vald(f_name.value, l_name.value, dob.value, room.value, ph_no.value, one_to_one.value)
    if p1.validate()==0:
        sql_qry = '''INSERT INTO patient(first_name,last_name,dob,room_no,emergency_contact,one_one_status,enrolement_status,details_id)VALUES(?,?,?,?,?,?,?,?);'''
        if one_to_one.value == 'No':
            one_one = 0
        else:
            one_one = 1
        #insertion in mongo db with error handling
        try:
            mg=MongoClient('localhost', 27017)
            mongo_db = mg.care_db_mongo
            patient_details = mongo_db.details 
            details_id = str(uuid.uuid1()) #generate unique id
            patient_details.insert_one({'_id':details_id,'schedule':str(schedule.value),'medical history':str(medical_history.value),'care plan':str(care_plan.value)})
        except:
            print("An issue occured while connecting with No-Sql database")
        data = (f_name.value, l_name.value, dob.value, room.value, ph_no.value, one_one,1,details_id)
        #insertion in SQLite with error handling
        try:
            db = "care_db.db"
            connection = sql.connect(db)
            cur = connection.cursor()
            cur.execute(sql_qry,data)
            connection.commit()
            connection.close()
            clear_output()
            print("\n \n Succesfully added new patient to database \n \n")
        except Exception as err:
            print('Database insertion failed: %s\nError: %s' % (sql_qry, str(err)))
            
button.on_click(button_click)

HTML(value='<h4><b>Enter patient details here</b></h4>')

Text(value='', description='First Name :', placeholder='Type first name')

Text(value='', description='Last Name :', placeholder='Type last name')

DatePicker(value=None, description='Date of birth :')

Text(value='', description='Room no :', placeholder='Enter room number')

...

HTML(value='<h4><b>Enter patient Care Plan here</b></h4>')

Textarea(value='', layout=Layout(flex='0 1 auto', height='100px', min_height='40px', width='auto'))

HTML(value='<h4><b>Enter patient Medical history here</b></h4>')

Textarea(value='', layout=Layout(flex='0 1 auto', height='100px', min_height='40px', width='auto'))

Button(button_style='info', description='Submit', icon='check', style=ButtonStyle(), tooltip='Confirm informat…

# Read patient details

In [21]:
#display patients inside drop box
db = "care_db.db"
connection = sql.connect(db)
cur = connection.cursor()
cur.execute('''SELECT patient_id,first_name,last_name,room_no FROM patient WHERE enrolement_status = 1''')
display_tags = []
id_tags = []
for i in cur.fetchall():
#     print(i)
    id_tags.append(i[0])
    display_tags.append(str(i[1])+" "+str(i[2]+" "+str(i[3])))

#define widgets
select = widgets.Dropdown(options=display_tags, value=  display_tags[0], description='Select Patient:')
button = widgets.Button(
    description='Go',
    disabled=False,
    button_style='',
    tooltip='Insert details for this patient',
    icon='' 
)
#button click event
def button_click(x):
    clear_output()
    p_id = id_tags[display_tags.index(select.value)]
    cur.execute('''SELECT first_name,last_name,dob,room_no,emergency_contact,one_one_status FROM patient WHERE patient_id = '''+str(p_id))
    for x in cur.fetchall():
        if x[5]==1:
            val = 'Yes'
        else:
            val = 'No'
        print(f'\n\n Name:{x[0]} {x[1]} \n DOB: {x[2]} \n Room no:{x[3]} \n Emergency contact {x[4]} \n One-One status {val}')
    details_id = cur.execute('''SELECT details_id FROM patient WHERE patient_id = '''+str(p_id))
    search_id = details_id.fetchone()[0]
    #search file in mongo db
    mg=MongoClient('localhost', 27017)
    mongo_db = mg.care_db_mongo
    patient_details = mongo_db.details
    document = patient_details.find_one({'_id': search_id})
    display( 
    widgets.HTML(
    value="<h4><b>Patient Schedule</b></h4>",
    ))
    print(document.get('schedule'))
    display( 
    widgets.HTML(
    value="<h4><b>Patient medical history</b></h4>",
    ))
    print(document.get('medical history'))
    display( 
    widgets.HTML(
    value="<h4><b>Patient care plan</b></h4>",
    ))
    print(document.get('care plan'))
    
    
display(select,button)
button.on_click(button_click)



 Name:dsf dfs 
 DOB: 2022-07-14 
 Room no:23 
 Emergency contact 12 
 One-One status No


HTML(value='<h4><b>Patient Schedule</b></h4>')

1221
d yoyoyoyyoyyoo


HTML(value='<h4><b>Patient medical history</b></h4>')

d21xxxxxxxxxxxxxxxx22
222222222222222222222222222222222222

32325


HTML(value='<h4><b>Patient care plan</b></h4>')

d2112-ooooop- jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj


# Update patient details

In [3]:
#display patients inside drop box
db = "care_db.db"
connection = sql.connect(db)
cur = connection.cursor()
cur.execute('''SELECT patient_id,first_name,last_name,room_no FROM patient WHERE enrolement_status = 1''')
display_tags = []
id_tags = []
for i in cur.fetchall():
#     print(i)
    id_tags.append(i[0])
    display_tags.append(str(i[1])+" "+str(i[2]+" "+str(i[3])))

#define widgets
select = widgets.Dropdown(options=display_tags, value=  display_tags[0], description='Select Patient:')
button = widgets.Button(
    description='Update',
    disabled=False,
    button_style='',
    tooltip='Update details for this patient',
    icon='' 
)
unregister_patient= widgets.Button(
    description='Unregister',
    disabled=False,
    button_style='warning',
    tooltip='Unregister this patient from care',
    icon='' 
)
one_one_stat= widgets.Button(
    description='Toggle one-one',
    disabled=False,
    button_style='info',
    tooltip='Change one-one status for this patient',
    icon='' 
)
change_shedule = widgets.Button(
    description='Change schedule',
    disabled=False,
    button_style='info',
    tooltip='Change schedule for this patient',
    icon='' 
)
update_schedule = widgets.Button(
    description='update',
    disabled=False,
    button_style='info',
    tooltip='update with this information',
    icon='' 
)
change_care = widgets.Button(
    description='Change Care plan',
    disabled=False,
    button_style='info',
    tooltip='Change Care plan for this patient',
    icon='' 
)
update_care = widgets.Button(
    description='update',
    disabled=False,
    button_style='info',
    tooltip='update with this information',
    icon='' 
)
change_medical = widgets.Button(
    description='Change Medical history',
    disabled=False,
    button_style='info',
    tooltip='Change Medical history for this patient',
    icon='' 
)
update_medical = widgets.Button(
    description='update',
    disabled=False,
    button_style='info',
    tooltip='update with this information',
    icon='' 
)

l = Layout(flex='0 1 auto', height='100px', min_height='40px', width='auto')
sch_area = Textarea(value='', layout=l)
care_area = Textarea(value='', layout=l)

temp = []
temp2_one_one = []
tem3_details = []

def button_click(x):
    clear_output()
    p_id = id_tags[display_tags.index(select.value)]
    temp.append(p_id)
    cur.execute('''SELECT first_name,last_name,room_no,one_one_status FROM patient WHERE patient_id = '''+str(p_id))
    data = cur.fetchall()[0]
    if data[3] == 1:
        val = 'Yes'
    else:
        val = 'No'
    print(f' Frist name: {data[0]} \n Last name: {data[1]} \n Room no: {data[2]} \n Currently on One-One: {val}')
    display(unregister_patient,"Note: Once unregistered you would have to contact database administrator to re-register",
            one_one_stat,change_shedule,change_care,change_medical)
    temp2_one_one.append(data[3])

def unregister(a):
    cur.execute('''UPDATE patient SET enrolement_status=0 WHERE patient_id = '''+str(temp[0]))
    connection.commit()
    connection.close()
    clear_output()
    print("\n \n Patient succesfuly unregistered from care \n \n ")

def one_one(b):
    if temp2_one_one[0] == 1:
        val = 0
    else:
        val = 1
    cur.execute('''UPDATE patient SET one_one_status='''+ str(val) +''' WHERE patient_id = '''+str(temp[0]))
    connection.commit()
    connection.close()
    clear_output()
    print("\n \n Patient One-One status change \n \n ")
#-------------------------------------------------------------------Updating schedule--------------------------------------------------------------------------
def sched(c):
    clear_output()
    cur.execute('''SELECT details_id FROM patient WHERE patient_id = '''+str(temp[0]))
    id_details = cur.fetchall()[0][0]
    tem3_details.append(id_details)
    try:
        mg=MongoClient('localhost', 27017)
        mongo_db = mg.care_db_mongo
        patient_details = mongo_db.details 
        sched_finder = patient_details.find_one({"_id": id_details})
    except:
        print("An issue occured while connecting with No-Sql database")
    print("Edit Schedule below")
    l = Layout(flex='0 1 auto', height='100px', min_height='40px', width='auto')
    sch_area = Textarea(value=str(sched_finder.get('schedule')), layout=l,disabled=False)
    display(sch_area,update_schedule)

    def confirm_sched(d):
        try:
            mg=MongoClient('localhost', 27017)
            mongo_db = mg.care_db_mongo
            patient_details = mongo_db.details
            newvalues = { "$set": { "schedule": str(sch_area.value) } }
            patient_details.update_one({"_id": tem3_details[0]}, newvalues)
            clear_output()
            print('Data has been updated')
        except:
            print("An issue occured while connecting with No-Sql database")
    update_schedule.on_click(confirm_sched)
#------------------------------------------------------------------Updating care plan--------------------------------------------------------------------------
def care(x):
    clear_output()
    cur.execute('''SELECT details_id FROM patient WHERE patient_id = '''+str(temp[0]))
    id_details = cur.fetchall()[0][0]
    tem3_details.append(id_details)
    try:
        mg=MongoClient('localhost', 27017)
        mongo_db = mg.care_db_mongo
        patient_details = mongo_db.details 
        sched_finder = patient_details.find_one({"_id": id_details})
    except:
        print("An issue occured while connecting with No-Sql database")
    print("Edit Care plan below")
    l = Layout(flex='0 1 auto', height='100px', min_height='40px', width='auto')
    sch_area = Textarea(value=str(sched_finder.get('care plan')), layout=l,disabled=False)
    display(sch_area,update_care)

    def confirm_care_fun(d):
        try:
            mg=MongoClient('localhost', 27017)
            mongo_db = mg.care_db_mongo
            patient_details = mongo_db.details
            newvalues = { "$set": { "care plan": str(sch_area.value) } }
            patient_details.update_one({"_id": tem3_details[0]}, newvalues)
            clear_output()
            print('Data has been updated')
        except:
            print("An issue occured while connecting with No-Sql database")
    update_care.on_click(confirm_care_fun)    
#--------------------------------------------------------------------Updating medical history----------------------------------------------------------------------
def med(x):
    clear_output()
    cur.execute('''SELECT details_id FROM patient WHERE patient_id = '''+str(temp[0]))
    id_details = cur.fetchall()[0][0]
    tem3_details.append(id_details)
    try:
        mg=MongoClient('localhost', 27017)
        mongo_db = mg.care_db_mongo
        patient_details = mongo_db.details 
        sched_finder = patient_details.find_one({"_id": id_details})
    except:
        print("An issue occured while connecting with No-Sql database")
    print("Edit Medical history below")
    l = Layout(flex='0 1 auto', height='100px', min_height='40px', width='auto')
    sch_area = Textarea(value=str(sched_finder.get('medical history')), layout=l,disabled=False)
    display(sch_area,update_medical)

    def confirm_medical(d):
        try:
            mg=MongoClient('localhost', 27017)
            mongo_db = mg.care_db_mongo
            patient_details = mongo_db.details
            newvalues = { "$set": { "medical history": str(sch_area.value) } }
            patient_details.update_one({"_id": tem3_details[0]}, newvalues)
            clear_output()
            print('Data has been updated')
        except:
            print("An issue occured while connecting with No-Sql database")
    update_medical.on_click(confirm_medical)  

#---------------------------------------------------------------------------------------------------------------------------------------

display(select,button)
button.on_click(button_click)
unregister_patient.on_click(unregister)
one_one_stat.on_click(one_one)
change_shedule.on_click(sched)
change_care.on_click(care)
change_medical.on_click(med)

 Frist name: tony 
 Last name: chako 
 Room no: 346 
 Currently on One-One: No


Button(button_style='warning', description='Unregister', style=ButtonStyle(), tooltip='Unregister this patient…

'Note: Once unregistered you would have to contact database administrator to re-register'

Button(button_style='info', description='Toggle one-one', style=ButtonStyle(), tooltip='Change one-one status …

Button(button_style='info', description='Change schedule', style=ButtonStyle(), tooltip='Change schedule for t…

Button(button_style='info', description='Change Care plan', style=ButtonStyle(), tooltip='Change Care plan for…

Button(button_style='info', description='Change Medical history', style=ButtonStyle(), tooltip='Change Medical…

# Delete patient details permenantly

In [20]:
db = "care_db.db"
connection = sql.connect(db)
cur = connection.cursor()
cur.execute('''SELECT patient_id,first_name,last_name,room_no,details_id FROM patient''')
display_tags = []
id_tags = []
details_tags = []

for i in cur.fetchall():
#     print(i)
    id_tags.append(i[0])
    display_tags.append(str(i[1])+" "+str(i[2]+" "+str(i[3])))
    details_tags.append(str(i[4]))
    
select = widgets.Dropdown(options=display_tags, value=  display_tags[0], description='Select Patient:')
button = widgets.Button(
    description='DELETE !',
    disabled=False,
    button_style='Warning',
    tooltip='Delete this patient from Database',
    icon='' 
)
def del_button(x):
    del_id = id_tags[display_tags.index(select.value)]
    del_details = details_tags[display_tags.index(select.value)]
    try:
        cur.execute('''DELETE FROM patient WHERE patient_id = '''+str(del_id))
        connection.commit()
        connection.close()
        mg=MongoClient('localhost', 27017)
        mongo_db = mg.care_db_mongo
        patient_details = mongo_db.details
        patient_details.delete_one({"_id": del_details})
        print("succesfully deleted patient details from all databases")
    except:
        print("A problem occured, check db")


display(select,button)
button.on_click(del_button)

Dropdown(description='Select Patient:', options=('Bonny bekham 125', 'tony chako 346', 'henry don 2', 'merlin …

Button(button_style='warning', description='DELETE !', style=ButtonStyle(), tooltip='Delete this patient from …

# INSERT carer details

In [12]:
button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='info',
    tooltip='Confirm and submmit',
    icon='check' 
)
f_name = widgets.Text(
placeholder='Type first name',
description='First Name :'
)
l_name = widgets.Text(
placeholder='Type last name',
description='Last Name :'
)
o_name = widgets.Text(
placeholder='Type organization name',
description='Org name :'
)
display(widgets.HTML(
    value="<h4><b>Enter carer details</b></h4>",
    ),f_name,l_name,o_name,button)

def button_fun(x):
    try:
        db = "care_db.db"
        connection = sql.connect(db)
        cur = connection.cursor()
        data = (f_name.value,l_name.value,o_name.value)
        sql_qry = '''INSERT INTO carer(first_name,last_name,org_name)VALUES(?,?,?);'''
        cur.execute(sql_qry,data)
        clear_output()
        print("\n \n Succesfully added new carer \n \n ")
        connection.commit()
        connection.close()
    except:
        print("An error occured while connecting database")
button.on_click(button_fun)


 
 Succesfully added new carer 
 
 


# Read carer details

In [20]:
db = "care_db.db"
connection = sql.connect(db)
cur = connection.cursor()
cur.execute('''SELECT * FROM carer ''')
for i in cur.fetchall():
    print(f' First name : {i[1]} {i[2]} \n Organization name: {i[3]} \n ------------------------------------------------------')
connection.commit()
connection.close()

 First name : Bob Cnya 
 Organization name: staff 
 ------------------------------------------------------
 First name : Diya C 
 Organization name: nurse plus 
 ------------------------------------------------------
 First name : Tommy dicoda 
 Organization name: staff 
 ------------------------------------------------------


# Delete carer details

In [24]:
db = "care_db.db"
connection = sql.connect(db)
cur = connection.cursor()
cur.execute('''SELECT carer_id,first_name,last_name,org_name FROM carer''')
display_tags = []
id_tags = []

for i in cur.fetchall():
#     print(i)
    id_tags.append(i[0])
    display_tags.append(str(i[1])+" "+str(i[2]+" "+str(i[3])))
    
select = widgets.Dropdown(options=display_tags, value=  display_tags[0], description='Select Carer:')
button = widgets.Button(
    description='DELETE !',
    disabled=False,
    button_style='Warning',
    tooltip='Delete this patient from Database',
    icon='' 
)

def del_button(x):
    del_id = id_tags[display_tags.index(select.value)]
    try:
        cur.execute('''DELETE FROM carer WHERE carer_id = '''+str(del_id))
        connection.commit()
        connection.close()
        print("succesfully deleted carer details from database")
    except:
        print("A problem occured, check db")


display(select,button)
button.on_click(del_button)

Dropdown(description='Select Carer:', options=('Bob Cnya staff', 'Diya C nurse plus', 'Tommy dicoda staff'), v…

Button(button_style='warning', description='DELETE !', style=ButtonStyle(), tooltip='Delete this patient from …

succesfully deleted patient details from all databases
